In [3]:
# import the libraries
import requests
from bs4 import BeautifulSoup

## Scraping all pages

In [1]:
# we'll be using the cars and conversations forum
url = 'https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations'

In [2]:
# number of forum pages
num_pages = 2572

In [4]:
# store the comments in a string
all_comments = ""

# go through each page
for page_number in range(1, num_pages + 1):
    # first page string of url isn't anything special
    try:
        if(page_number == 1):
            new_url = url
        else:
            # end url format is /p220 etc.
            new_url = url + "/p" + str(page_number)
        print(new_url)
        # collect the url with requests library
        page = requests.get(new_url)
    
        # get the html of the page in string form
        page_html = page.text
    
        # create the BeautifulSoup object that takes in the html in str form and a html/xml parser of choice either html.parser or lxml
        soup = BeautifulSoup(page_html, 'html.parser')
    
        # inspect the elements from the webpage to know where stuff is
        # in this case the comments are under a <p> tag which are all under a <div class = "Message userContent"</div/ tag 
        # which are all under a <div class = "MessageList DataList Comments"> tag
    
        # Pull all text from the 'MessageList DataList Comments' divs
        messagelist = soup.find(class_ = "MessageList DataList Comments")
    
        # within the 'MessageList DataList Comments' divs pull all text from 'Message userContent' divs
        usermessages = messagelist.find_all(class_ =  "Message userContent")
    
        # now extract just the <p> tags from all comments! 
        for i in range(len(usermessages)):
            # a user comment might have multiple <p> tags
            [s.extract() for s in usermessages[i](class_ = 'UserQuote')]
            user_comments = usermessages[i].find_all('p')
            for j in range(len(user_comments)):
                # remove <a>, <img>, <br> tags embedded in <p>
                [s.extract() for s in user_comments[j]('a')]
                [s.extract() for s in user_comments[j]('img')]
                [s.extract() for s in user_comments[j]('br')]

            
                # add the users comments to the mega string
                all_comments += user_comments[j].prettify()
    
        # print some checks
        print('Page ' + str(page_number) + ' complete')
    except:
        print('page ' + str(page_number) + ' failed')
        continue

https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations
Page 1 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2
Page 2 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p3
Page 3 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p4
Page 4 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p5
Page 5 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p6
Page 6 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p7
Page 7 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p8
Page 8 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p9
Page 9 complete


In [75]:
print(all_comments[:1000])

<p>
 I'm 2nd post....yee haw!!!!!!!!
</p>
<p>
 Do I get a badge?
</p>
<p>
 Great, but will it be possible to refer to old thread?
</p>
<p>
 And one more thing how do I find and add a discussion to "My Bookmarks?"
</p>
<p>
 If you have posted in that discussion, click on "Participated" and all your past post topics are chronologically listed.
</p>
<p>
 If you are looking for a brand or model or a keyword, use the "Car forums" link at the top left of this page to get some ways of searching for models, brands, keywords, or poster names.
</p>
<p>
 if you want to bookmark this one, there is a little 5 point star next to the title up above.  just click that and it turns yellow.
</p>
<p>
 Thanks
 , it worked like a charm.
</p>
<p>
 Thanks imidazol97. I will try that later but stickguy's approach works for me.
</p>
<p>
 Thanks for that. While this new board loads faster it just seems a little cumbersome and not very user friendly.
</p>
<p>
 No, but it's almost like you're famous.  Just think, 

In [76]:
# let's do some string manipulation
# remove the newline characters, '<p>', '</p','said:'
parsedData = all_comments.replace('\n', '')
parsedData = parsedData.replace(r"<p>", "")
parsedData = parsedData.replace(r"</p>", "")
parsedData = parsedData.replace(r"</p>", "")
parsedData = parsedData.replace(r"said:", "")
parsedData = parsedData.replace(r":", "")
parsedData = parsedData.lower()

In [72]:
parsedData

' i\'m 2nd post....yee haw!!!!!!!! do i get a badge? great, but will it be possible to refer to old thread? and one more thing how do i find and add a discussion to "my bookmarks?" if you have posted in that discussion, click on "participated" and all your past post topics are chronologically listed. if you are looking for a brand or model or a keyword, use the "car forums" link at the top left of this page to get some ways of searching for models, brands, keywords, or poster names. if you want to bookmark this one, there is a little 5 point star next to the title up above.  just click that and it turns yellow. thanks , it worked like a charm. thanks imidazol97. i will try that later but stickguy\'s approach works for me. thanks for that. while this new board loads faster it just seems a little cumbersome and not very user friendly. no, but it\'s almost like you\'re famous.  just think, if this discussion goes on for years and years like the other did, visitors loading this discussion 

In [77]:
# throw this into a text file just in case
new_file = open('alltext.txt', 'w', encoding = 'utf-8')
new_file.write(parsedData)
new_file.close()

In [78]:
# convert model names to brands
import csv

# open the models file and use that to replace model names with brand names. 
# format of file is brand, model
file = open('models.csv', 'r')
csv_file = csv.reader(file)

# store the brand names
brands = []
for i in csv_file:
    parsedData = parsedData.replace(i[1], i[0])
    if(i[0] not in brands):
        brands.append(i[0])

print('Finished converting model names to brands')
# close the file
file.close()

Finished converting model names to brands


## Word Frequency

In [79]:
# grab word frequency using nltk library
import nltk
# stop words
from nltk.corpus import stopwords
nltk.download()
nltk.download('stopwords')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [80]:
# tokenize the text 
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(parsedData)

In [81]:
# let's do a word count
from collections import Counter

# counter object
words = Counter()

# update counter with new words
words.update(tokens)

In [82]:
# check
words.most_common()[:100]

[('the', 9528),
 ('i', 6021),
 ('a', 5921),
 ('to', 4990),
 ('and', 4469),
 ('it', 3346),
 ('of', 3145),
 ('that', 2866),
 ('in', 2759),
 ('is', 2426),
 ('for', 2266),
 ('you', 2202),
 ('car', 1860),
 ('on', 1768),
 ('t', 1609),
 ('was', 1567),
 ('they', 1524),
 ('with', 1492),
 ('s', 1456),
 ('my', 1443),
 ('but', 1435),
 ('have', 1428),
 ('are', 1116),
 ('be', 1102),
 ('as', 1003),
 ('at', 987),
 ('not', 962),
 ('or', 926),
 ('if', 921),
 ('one', 860),
 ('he', 835),
 ('like', 823),
 ('so', 792),
 ('just', 777),
 ('would', 771),
 ('about', 768),
 ('we', 749),
 ('me', 740),
 ('this', 711),
 ('can', 708),
 ('had', 693),
 ('what', 686),
 ('an', 657),
 ('all', 642),
 ('there', 635),
 ('out', 626),
 ('get', 625),
 ('up', 621),
 ('will', 614),
 ('from', 599),
 ('more', 574),
 ('when', 562),
 ('some', 560),
 ('your', 552),
 ('don', 540),
 ('new', 539),
 ('do', 511),
 ('has', 510),
 ('think', 502),
 ('their', 468),
 ('them', 464),
 ('good', 460),
 ('no', 448),
 ('were', 427),
 ('by', 402),
 (

So these are the most common words, but they are also the same in any english text. Therefore they are not very insightful. Hence, let's remove single characters, numbers, and common words.

In [83]:
# store the stop words
stopwords = stopwords.words('english')[:]

In [84]:
# lets do this again: remove stop words, single letter variable names
words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1)

In [85]:
# check
words.most_common()[:1000]

[('car', 1860),
 ('one', 860),
 ('like', 823),
 ('would', 771),
 ('get', 625),
 ('new', 539),
 ('think', 502),
 ('good', 460),
 ('back', 387),
 ('years', 372),
 ('know', 367),
 ('time', 366),
 ('people', 362),
 ('dealer', 345),
 ('ford', 342),
 ('much', 342),
 ('could', 335),
 ('well', 318),
 ('buy', 312),
 ('even', 310),
 ('benz', 299),
 ('see', 297),
 ('year', 285),
 ('got', 285),
 ('really', 277),
 ('want', 276),
 ('said', 272),
 ('go', 261),
 ('price', 255),
 ('way', 254),
 ('going', 252),
 ('make', 244),
 ('still', 238),
 ('many', 229),
 ('bmw', 229),
 ('better', 228),
 ('also', 227),
 ('money', 225),
 ('never', 222),
 ('right', 220),
 ('used', 217),
 ('old', 213),
 ('gm', 211),
 ('day', 203),
 ('last', 203),
 ('lot', 199),
 ('take', 197),
 ('nice', 193),
 ('first', 191),
 ('two', 189),
 ('problem', 189),
 ('drive', 185),
 ('something', 182),
 ('sure', 179),
 ('may', 179),
 ('every', 179),
 ('miles', 176),
 ('probably', 174),
 ('around', 173),
 ('us', 172),
 ('use', 169),
 ('look'

In [86]:
# now let's do this again but with only brand names
words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1 and x in brands)

In [87]:
brands

['acura',
 'audi',
 'bmw',
 'buick',
 'cadillac',
 'car',
 'chevrolet',
 'chrysler',
 'dodge',
 'ford',
 'honda',
 'hyndai kia',
 'hyundai',
 'hyundai,',
 'hyundai.',
 'infiniti',
 'kia',
 'kia,',
 'kia.',
 'lincoln',
 'mazda',
 'mercedes',
 'mercedes benz',
 'mercedes-benz',
 'mercury',
 'mitsubishi',
 'nissan',
 'nissan.',
 'pontiac',
 'problem',
 'saturn',
 'seat',
 'sedan',
 'subaru',
 'suzuki',
 'toyata',
 'toyota',
 'volkswagen',
 'volkwagen',
 'volvo']

In [88]:
# check
words.most_common()[:]

[('car', 1860),
 ('ford', 342),
 ('bmw', 229),
 ('problem', 189),
 ('mercedes', 149),
 ('chevrolet', 148),
 ('buick', 136),
 ('audi', 129),
 ('volkswagen', 125),
 ('honda', 105),
 ('toyota', 102),
 ('seat', 100),
 ('cadillac', 81),
 ('dodge', 80),
 ('chrysler', 77),
 ('acura', 67),
 ('subaru', 57),
 ('nissan', 55),
 ('lincoln', 54),
 ('hyundai', 49),
 ('mazda', 44),
 ('sedan', 40),
 ('volvo', 39),
 ('kia', 19),
 ('saturn', 8),
 ('pontiac', 7),
 ('infiniti', 6),
 ('mitsubishi', 3),
 ('mercury', 2)]